In [2]:
## Import relavant librarys

# Data librarys
import numpy as np
import pandas as pd

# Ploting library
import matplotlib.pyplot as plt
%matplotlib inline

In [228]:
def checkNAN(pandasdf):
    return pd.isnull(pandasdf).any(1).nonzero()[0]

In [181]:
## Readin CSV data using pandas 
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

m_data = train.shape[0]
m_test = test.shape[0]
train_valid_ratio = 0.8
print('Shape of the training data = '+str(train.shape))
print('Shape of the test data = '+str(test.shape))

## Append data together
print('type of the train data - ' + str(type(train)))
full = train.append(test, ignore_index=True)
print('Shape of the full data = ' + str(full.shape))

train = full


Shape of the training data = (891, 12)
Shape of the test data = (418, 11)
type of the train data - <class 'pandas.core.frame.DataFrame'>
Shape of the full data = (1309, 12)


In [167]:
train.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [168]:
train.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [169]:
## Transform Pclass
pclass = pd.get_dummies(train.Pclass)

## Transform Sex into male:feamle = 1:0
sex = pd.Series(np.where(train.Sex=='male',0,1))
#sex = pd.get_dummies(train.Sex)

## Tranform embarked type CQS into indices
embarked = pd.get_dummies(train.Embarked)

## Tranform cabin type
cabin = train.Cabin.fillna('U')
cabin = cabin.map(lambda s:s[0])
cabin=pd.get_dummies(cabin)
#print(cabin)
## Transform famly size
family = train.Parch+train.SibSp+1
family = pd.DataFrame(np.where(parch>1,1,0))

In [170]:
## Obtain of people
name = train.Name
# we extract the title from each name
titles = name.map(lambda name: name.split(',')[1].split('.')[0].strip())
titles = pd.get_dummies(titles)

In [229]:
## Transform ticket 
# only use the number in the ticket..
#   separate ticket in two stations and obained order based on number of digits
tickets = train.Ticket
def getTicketStation(ticket):
    tmp = ticket.split(' ')
    if len(tmp)>1:
        return tmp[0][0]
    else:
        return 'U_station'
def getTicketNumber(ticket):
    tmp = ticket.split(' ')
    if len(tmp)>1:
        return len(tmp[1])
    else:
        return len(tmp[0])
    
stations = pd.get_dummies(tickets.map(getTicketStation))
#np.where(tickets=='mp',1,0)
ticketNumbers = tickets.map(getTicketNumber)
ticketNumbers = pd.get_dummies(ticketNumbers)
#ticketnumbers
#stations


In [203]:
## Selecting a model from SKlearn
##
##
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

In [248]:
## Check for NAN
print(checkNAN(pclass), sex.isnull().all(),checkNAN(embarked),checkNAN(cabin), checkNAN(family), checkNAN(titles), \
      checkNAN(stations), checkNAN(ticketNumbers))

(array([], dtype=int64), False, array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [249]:
full_x = pd.concat([pclass, sex, embarked, cabin, family, titles, stations, ticketNumbers], axis=1)
m_train = int(m_data*train_valid_ratio)

train_x , valid_x , train_y , valid_y = train_test_split( full_x[0:m_data] , train.Survived[0:m_data] , \
                                                            train_size = train_valid_ratio )
#train_x = full_x[0:m_train]
#train_y = train.Survived[0:m_train]

#valid_x = full_x[m_train:m_data]
#valid_y = train.Survived[m_train:m_data]

test_x = full_x[m_data:(m_data+m_test)]

print(train_x.shape, valid_x.shape, test_x.shape, train.shape)

((712, 50), (179, 50), (418, 50), (1309, 12))


In [250]:
# Fit the training set
model.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [251]:
### Yeah! my first training!
print (model.score( train_x , train_y ) , model.score( valid_x , valid_y ))

(0.8314606741573034, 0.81564245810055869)


In [259]:
#np.where(test_x==np.nan,1,0)
type(test_x)
print(checkNAN(test_x))
test_x.reset_index()
test_y = model.predict(test_x)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').